In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

In [48]:
from zipfile import ZipFile

In [49]:
with ZipFile('task.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [50]:
d = 'Activity Recognition from Single Chest-Mounted Accelerometer'
PATH = '../'

In [51]:
os.path.join(os.getcwd(),'Activity Recognition from Single Chest-Mounted Accelerometer/')

'/home/akku/Desktop/myenv/Activity Recognition from Single Chest-Mounted Accelerometer/'

In [52]:
data_dir=os.path.join(os.getcwd(),'Activity Recognition from Single Chest-Mounted Accelerometer/')

In [53]:
files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]


In [54]:
s_cols = ['0', '1866', '2390', '2282', '1', 'user']

In [55]:
data=pd.DataFrame()
#s_cols=list(df.columns)
#s_cols = s_cols + ["id"]
id=0
ids=[]
for file_name in  files:
        id +=1
        df = pd.read_csv(data_dir + str(file_name))
        df["user"] =id
        df.columns=s_cols
        data=pd.concat([data,df],axis=0)
        ids.append(id)
        

In [56]:
s_cols=list(df.columns)

In [57]:
data.columns=(["sequential number", "x acceleration", "y acceleration", "z acceleration", "label","user"])

In [58]:
activities = {
        1: "Working at Computer",
        2: "Standing Up, Walking and Going updown stairs",
        3: "Standing",
        4: "Walking",
        5: "Going UpDown Stairs",
        6: "Walking and Talking with Someone",
        7: "Talking while Standin",        
    }

In [59]:
data['activity_1']=data.label.map(activities)

In [60]:
# removing the activity column

x = data.drop(['label','sequential number','user'], axis = 1)

y=data['label']


In [61]:
from sklearn.model_selection import train_test_split as tt
x_train,x_test,y_train,y_test=tt(x,y,test_size=0.20,random_state=0)

In [62]:
from sklearn.preprocessing import RobustScaler
scale_columns = ['x acceleration', 'y acceleration', 'z acceleration']

scaler = RobustScaler()

scaler = scaler.fit(x_train[scale_columns])

x_train.loc[:, scale_columns] = scaler.transform(
  x_train[scale_columns].to_numpy()
)

x_test.loc[:, scale_columns] = scaler.transform(
  x_test[scale_columns].to_numpy()
)

/home/akku/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/home/akku/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [63]:
from scipy import stats
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [64]:
TIME_STEPS = 200
STEP = 40

X_train, y_train = create_dataset(
    x_train[['x acceleration', 'y acceleration', 'z acceleration']],
    x_train.activity_1,
    TIME_STEPS,
    STEP
)

X_test, y_test = create_dataset(
    x_test[['x acceleration', 'y acceleration', 'z acceleration']],
    x_test.activity_1,
    TIME_STEPS,
    STEP
)

In [65]:
print(X_train.shape, y_train.shape)


(38533, 200, 3) (38533, 1)


In [66]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

In [67]:
import keras

model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=128,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['acc']
)

In [71]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

Train on 34679 samples, validate on 3854 samples
Epoch 1/20
34679/34679 [==============================] - 343s 10ms/step - loss: 0.6975 - acc: 0.5348 - val_loss: 0.6973 - val_acc: 0.5246
Epoch 2/20
34679/34679 [==============================] - 344s 10ms/step - loss: 0.6968 - acc: 0.5359 - val_loss: 0.6972 - val_acc: 0.5246
Epoch 3/20
34679/34679 [==============================] - 344s 10ms/step - loss: 0.6969 - acc: 0.5352 - val_loss: 0.6975 - val_acc: 0.5246
Epoch 4/20
34679/34679 [==============================] - 345s 10ms/step - loss: 0.6968 - acc: 0.5369 - val_loss: 0.6963 - val_acc: 0.5246
Epoch 5/20
34679/34679 [==============================] - 345s 10ms/step - loss: 0.6964 - acc: 0.5372 - val_loss: 0.6965 - val_acc: 0.5246
Epoch 6/20
34679/34679 [==============================] - 363s 10ms/step - loss: 0.6966 - acc: 0.5371 - val_loss: 0.6965 - val_acc: 0.5246
Epoch 7/20
34679/34679 [==============================] - 345s 10ms/step - loss: 0.6964 - acc: 0.5368 - val_loss: 0.6

In [69]:
model.evaluate(X_test, y_test)

9630/9630 [==============================] - 24s 3ms/step


[0.6987705431251883, 0.5304257273674011]

In [70]:
y_pred = model.predict(X_test)